In [10]:
import requests

resp = requests.post("http://localhost:7071/api/getNodeDetails", json={"node_id": "integrals"})
print(resp.json())

{'topic': ['Integrals'], 'status': ['unstarted'], 'pk': ['pk']}


In [5]:
from enum import Enum
test = {"test":1, "test2":2}
print("\n- ".join(test.keys()))

test
- test2


In [65]:
import ast
from typing import List, Optional, Type
import azure.functions as func
from psycopg2 import pool
from pydantic import BaseModel, Field, ValidationError, root_validator, validator, ConfigDict
from enum import Enum
from langchain_openai import AzureChatOpenAI
from langchain_core.runnables import RunnableLambda
from langchain.agents.output_parsers.openai_tools import OpenAIToolsAgentOutputParser
from langchain.tools.render import format_tool_to_openai_tool
from langchain.agents.format_scratchpad.openai_tools import format_to_openai_tool_messages
from langchain.output_parsers import OutputFixingParser, PydanticOutputParser
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.schema import StrOutputParser
from langchain.tools import BaseTool
import shutil
from langchain.callbacks.manager import (
    AsyncCallbackManagerForToolRun,
    CallbackManagerForToolRun,
)
from operator import itemgetter
import re
import json
import os
import subprocess
from ansi2html import Ansi2HTMLConverter
from langchain.agents import AgentExecutor

from langchain.callbacks.manager import (
    AsyncCallbackManagerForToolRun,
    CallbackManagerForToolRun,
)
from langgraph.prebuilt import ToolExecutor, ToolInvocation
from langgraph.graph import StateGraph, END
from langgraph.pregel import Pregel

# ROTATE THIS KEY
os.environ["OPENAI_API_KEY"] = "10388fa13cbd4ca0aa189687aa4b3af1"
os.environ['AZURE_OPENAI_ENDPOINT'] = "https://gpt-4-beta-canada.openai.azure.com/"

gpt_4_llm = AzureChatOpenAI(deployment_name="gpt-4-turbo", api_version="2023-07-01-preview", model_name="gpt-4-1106-preview", temperature=0, max_retries=10)

class CodeGenerateSchema(BaseModel):
    manim_code: str = Field(description="Manim code that generates the visuals for the scene. Treat this field as the equivalent to a .py file; it must be NOTHING BUT Python code and be able to be executed with manim -pql scene.py <class_name> AS-IS.")
    class_name: str = Field(description="The Manim scene class to use when running the manim -pql scene.py <class_name> command to render the video")
    desired_visual: str = Field(description="The exact input the user gave you describing the visual they want you to create")

    @root_validator
    def is_valid_code(cls, v):
        try:
            parsed_module = ast.parse(v['manim_code'])
            class_names = [node.name for node in ast.walk(parsed_module) if isinstance(node, ast.ClassDef)]
            if not any([v['class_name'] in name for name in class_names]):
                raise ValueError("Target render class not found in code!")
        except SyntaxError:
            raise ValueError("Code is not valid Python")
        return v
    
class CodeIssue(BaseModel):
    issue: str = Field(description="""A description of what issue with the animation this code will cause. Only report issues that fall into the below categories. Don't report the category itself, just what the actual issue is in the animation (e.g. the end of the pendulum moves at a different rate from the spring, causing the two to separate)
                       - issues that cause the animation to be choppy
                       - issues that cause elements to move out of the video and be cut off or not visible
                       - issues that cause elements to incorrectly overlap
                       - issues that cause animations to render off-center in the video
                       - issues that cause elements that should move as 1 component to move at different speeds or in different directions
                       - instances where movement that should be determined by physical equations is not determined by those equations (e.g., a pendulum that doesn't move according to the pendulum equation)
                       - issues that cause latex code will render as plain text with markup visible
                       - issues that cause elements to render larger or smaller than they should be for comfortable viewing
                       - issues where an animation is too long or short to voice over (e.g., a pendulum swinging for an entire minute or only once) UNLESS THE USER SPECIFICALLY REQUESTS IT""")
    code: str = Field(description="The code that causes the issue")
    improvement: str = Field(description="Code to replace the code field that will fix the issue. If the issue is not fixable, DON'T REPORT THE ISSUE AT ALL")

class CodeCorrectnessReport(BaseModel):
    issues: List[CodeIssue] = Field(description="A list of issues with the code that will cause the animation to differ from the user's desired visual. DO NOT include any issues in this list that do not match the categories in the issue field of the CodeIssue model. If there are no such issues, this field should be an empty list")
    
code_parser = PydanticOutputParser(pydantic_object=CodeCorrectnessReport)
code_fixing_parser = OutputFixingParser.from_llm(parser=code_parser, llm=gpt_4_llm)

class VideoGenerate(BaseTool):
    name = "RenderVideo"
    description = "Attempts to render the provided code into a video file and provides the stdout and stderr of the render process"
    args_schema: Type[BaseModel] = CodeGenerateSchema
    conv = Ansi2HTMLConverter()

    model_config = ConfigDict(from_attributes=True)

    def _run(self, manim_code: str, class_name: str, desired_visual: str, run_manager: Optional[CallbackManagerForToolRun] = None) -> str:
        """generate a video synchronously."""

        # get rid of old generation stuff
        if os.path.isfile("scene.py"):
            os.remove("scene.py")
        if os.path.isdir("media"):
            shutil.rmtree("media")

        # write the code to a file
        with open("scene.py", "w") as scene:
            scene.write(manim_code)

        stdout, stderr = subprocess.Popen(f"manim -pql scene.py {class_name}", stderr=subprocess.PIPE, stdout=subprocess.PIPE, shell=True).communicate()
        stdout_decoded = stdout.decode('utf-8')
        stderr_decoded = stderr.decode('utf-8')

        stdout_html = self.conv.convert(stdout_decoded)
        stderr_html = self.conv.convert(stderr_decoded)

        resp = {}

        # video rendered successfully; check for correctness
        if "File</span> ready at" in stdout_html:
            # file rendered successfully
            code_checking_prompt = ChatPromptTemplate.from_messages(
                [
                    ("system", "You are an agent whose job it is to grade how well a user's manim code generates their desired visual. The user will provide you with your code, and then the visual they are trying to generate. Ignore the educational context of the video entirely. You are only focusing on how accurate the video is to the user's desired visual. Return your answer in the following format: {format_instructions}"),
                    ("user", '{manim_code}\n\nWhen this manim code is rendered the user wants the following to happen: "{desired_visual}"\nDoes this code achieve this? Point out any specific code flaws that will cause the resulting animation to differ from that the user requested. DO NOT report any issue with the educational impact of the animation. For example if damping is not accounted for and the desired visual doesn\'t include damping, DO NOT say that damping should be added. If the video accurately creates the animation requested, tell the user that no further modifications are required.'),
                ]
            )
            code_checking_chain = (
                {
                    "format_instructions": itemgetter("format_instructions"),
                    "manim_code": itemgetter("manim_code"),
                    "desired_visual": itemgetter("desired_visual"),
                }
                | code_checking_prompt
                | gpt_4_llm
                | code_fixing_parser
            )

            code_correctness_output = code_checking_chain.invoke({
                "format_instructions": code_fixing_parser.get_format_instructions(),
                "manim_code": manim_code,
                "desired_visual": desired_visual
            })

            if code_correctness_output.issues == []:
                resp["correctness_report"] = "The code accurately generates the desired visual. No further action is needed"
            else:
                resp["correctness_report"] = code_correctness_output.dict()
        else:
            resp["stdout"] = stdout_html
            resp["stderr"] = stderr_html


        # if "See log output above or the log file:\n" in console_response["stderr"]:
        #     try:
        #         log_url = re.compile(r"\w+\/\w+\/[\w\d]+\.log").search(console_response["stderr"]).group(0)
        #         with open(log_url, "r") as log:
        #             log_contents = log.read()

        #         console_response["latex_log"] = log_contents
        #     except:
        #         print("couldn't get latex log")

        # See log output above or the log file:\nmedia/Tex/9587f5ca887a1ae0.log

        return json.dumps(resp)
        
    def _arun(self, run_manager: Optional[AsyncCallbackManagerForToolRun] = None) -> str:
        """Generate a video segment asynchronously."""
        raise NotImplementedError("TextGenerate does not support async")
    
# INCLUDE PROMPTING STYLE IN TEACHING EFFECTIVENESS REPORT. LACK OF ATTENTION CAN BE BECAUSE OF SHITTY VISUALS
    
tools = [VideoGenerate()]
tools_oai = [format_tool_to_openai_tool(tool) for tool in tools]
gpt4_tools = gpt_4_llm.bind(tools=tools_oai)

workflow = StateGraph(TaskRunState)
workflow.add_node("agent", self.call_model)
workflow.add_node("action", self.call_tool)
workflow.set_entry_point("agent")
workflow.add_conditional_edges(
    "agent",
    self.should_continue,
    {
        "continue": "action",
        "end": END
    }
)
workflow.add_edge("action", "agent")
app = workflow.compile()

code_creation_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are an agent whose job it is to write a python file which can be run using manim -pql scene.py ClassName to generate the animation or visual that the user requests. Don't respond with anything except for the tool call and a 1 sentence explanation of what you did or are fixing. Don't keep trying something that isn't working; instead, switch to a different approach. Each time you generate a video, you will get back the stderr if there is an error generating a video or a correctness_report if the video generates but there are code changes you need to make. If the video is perfect, you will be told that no further changes need to be made."),
        ("user", "{desired_animation}"),
        MessagesPlaceholder(variable_name="agent_scratchpad"),
    ]
)
code_creation_chain = (
    {
        "desired_animation": itemgetter("desired_animation"),
        "agent_scratchpad": itemgetter("intermediate_steps") | RunnableLambda(format_to_openai_tool_messages),
    }
    | code_creation_prompt
    | gpt_4_llm.bind(tools=tools_oai)
    | OpenAIToolsAgentOutputParser()
).with_retry()

agent = AgentExecutor(agent=code_creation_chain, tools=tools, verbose=True)

# 1. get basic video working (animations, equations, text)
# 2. incorporate visuals
# 3. incorporate plugins

# Create a visual of a spring oscillating, moving according to Hooke's Law. The spring should be attached to a wall on one end and a mass on the other. The mass should be displaced from its equilibrium position and then released. The mass should oscillate back and forth around the equilibrium position. The spring should be shown to stretch and compress as the mass moves. The mass should be shown to move with simple harmonic motion. The spring should be shown to obey Hooke's Law. The mass should be shown to obey Newton's Second Law. The mass should be shown to have kinetic energy and potential energy. The spring should be shown to have elastic potential energy. The mass should be shown to have maximum kinetic energy at the equilibrium position and maximum potential energy at the maximum displacement from the equilibrium position. The spring should be shown to have maximum elastic potential energy at the maximum displacement from the equilibrium position and maximum kinetic energy at the equilibrium position.
# Create an animation of 2-d Hooke's Law with a square with arrows pointing up and to the right for the x and y components of the force. Then fade in the equation for the x component, epsilon_x=sigma_x/E-v*sigma_y/E, and the equation for the y component, epsilon_y=sigma_y/E-v*sigma_x/E, one after the other

code_output = agent.invoke({
    "desired_animation": "Create a visual of a spring oscillating, moving according to Hooke's Law. The spring should be attached to a wall on one end and a mass on the other. The mass should be displaced from its equilibrium position and then released. The mass should oscillate back and forth around the equilibrium position. The spring should be shown to stretch and compress as the mass moves. The mass should be shown to move with simple harmonic motion. The spring should be shown to obey Hooke's Law.",
})

# "Create an animation of projectile motion with a ball being thrown off of a building towards the ground. Show the components of motion with arrows in the x and y direction from the ball that grow and shrink in proportion to how much each component of velocity is changing as the ball travels. Also, include text for the equations of motion, counters for the actual x and y components of velocity, and the height of the building in metric units."
# "Create an animation of a ball being launched off of a building and falling to the ground, following projectile motion. "



> Entering new AgentExecutor chain...

Invoking: `RenderVideo` with `{'manim_code': "from manim import *\n\n\nclass OscillatingSpring(Scene):\n    def construct(self):\n        # Constants for the spring and mass\n        spring_length = 3\n        mass = 2\n        k = 2  # Spring constant\n\n        # Create the wall, spring, and mass\n        wall = Square(side_length=0.5)\n        wall.to_edge(LEFT, buff=0)\n        spring = Spring(start=wall.get_right(), length=spring_length)\n        block = Square(side_length=1).set_fill(BLUE, opacity=1)\n        block.next_to(spring, RIGHT)\n\n        # Group the spring and block together\n        spring_mass_group = VGroup(spring, block)\n\n        # Function for Hooke's law\n        def update_spring(spring):\n            spring.put_start_and_end_on(wall.get_right(), block.get_center())\n\n        # Add updater to the spring\n        spring.add_updater(update_spring)\n\n        # Animate the mass oscillating\n        self.play(ApplyWave(blo

KeyboardInterrupt: 